In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb


In [13]:
# load dataset
max_features = 10000 #vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [4]:
# inspect data
sample_review  = X_train[0]
sample_label   = y_train[0]
print(len(X_train[0]))
print("Sample review (encoded): ", sample_review)
print("Sample label: ", sample_label)

218
Sample review (encoded):  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label:  1


In [41]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [6]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]],
      shape=(25000, 500), dtype=int32)

In [28]:
dimension = 128
model = Sequential()
model.add(Embedding(max_features, dimension, input_length=max_len))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

c:\Users\acer\OneDrive\Desktop\DeepLearning Nlp\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_6 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
from tensorflow.keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor = ' val_loss', patience = 5, restore_best_weights = True, mode = 'min')


In [46]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
print(X_train.dtype)

object


In [47]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    callbacks=[earlyStopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.7930 - loss: 258.2206 - val_accuracy: 0.5880 - val_loss: 0.7568
Epoch 2/10
  2/625 ━━━━━━━━━━━━━━━━━━━━ 40s 65ms/step - accuracy: 0.5312 - loss: 0.8089 

c:\Users\acer\OneDrive\Desktop\DeepLearning Nlp\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric ` val_loss` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


625/625 ━━━━━━━━━━━━━━━━━━━━ 57s 92ms/step - accuracy: 0.8304 - loss: 0.3874 - val_accuracy: 0.7976 - val_loss: 0.4421
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 66s 105ms/step - accuracy: 0.8985 - loss: 0.2511 - val_accuracy: 0.8046 - val_loss: 0.4424
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 105ms/step - accuracy: 0.9284 - loss: 0.1902 - val_accuracy: 0.8136 - val_loss: 0.4495
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 128ms/step - accuracy: 0.9463 - loss: 0.1499 - val_accuracy: 0.8234 - val_loss: 0.5019
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 156ms/step - accuracy: 0.9611 - loss: 0.1133 - val_accuracy: 0.8318 - val_loss: 0.5053
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 107s 171ms/step - accuracy: 0.9605 - loss: 0.1168 - val_accuracy: 0.7940 - val_loss: 0.6077
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 153ms/step - accuracy: 0.9753 - loss: 0.0767 - val_accuracy: 0.7734 - val_loss: 0.6521
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 100s 160ms/step - accuracy: 0.9782 - loss: 0.0675 - v

In [49]:
# saving the model
model.save("simplernn_imdb_model.h5")